In [890]:

#functions to deal with creating and extracting features

#import bamboolib as bam

# import bamboolib as bam
# import bamboolib as bam
from datetime import datetime, timedelta
#from VBTT2_IO.IO import  read_config_file

# import bamboolib as bam
import numpy as np
import pandas as pd
from yahoo_fin import stock_info as si


def get_yf_dataframe(data, nbdays):
    yesterday = datetime.now()  # we want data up to yesterday
    start_date = yesterday - timedelta(days=nbdays)  # we run the model using data for nbdays
    df_res = pd.DataFrame()
    #print("data in module get_yf_dataframe", data)
    #print("nbdays in  in module get_yf_dataframe", nbdays)
    for ticker in data:
        df_tmp = si.get_data(ticker, start_date, yesterday)
        df_res[ticker] = df_tmp['close']
    return df_res





In [891]:
def preprocessing(ticker, additional_data, days):
    #this function return a matrix of features augmented with fix data for number of days
    print ("preprocing module ticker",ticker)
    print("preprocessing module additional data",additional_data)
    print("preprocessing module days", days)
    tickers_in_sector_extended = np.concatenate((ticker, additional_data), axis=None)
    tickers_in_sector_extended = tickers_in_sector_extended.tolist()
    matrix_features_sector = get_yf_dataframe(tickers_in_sector_extended, days)
    # import pandas as pd; import numpy as np
    # matrix_features_sector = matrix_features_sector.reset_index()
    matrix_features_sector = matrix_features_sector.reindex(sorted(matrix_features_sector.columns), axis=1)

    ###################################
    ##### saving and reading features - can help increase processing time
    ##### to evaluate on future version
    ####################################
    # matrix_features_sector.to_csv("matrix_features_sector.csv")
    # matrix_features_sector=pd.read_csv("matrix_features_sector.csv")

    return matrix_features_sector

In [892]:

def create_train_test_set(ticker, features, lags,additional_data,days,nb_predict_days):
    # creating train and test set
    yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last, days = initialize_data(
        days,
        lags,
        nb_predict_days)

    df = features[[ticker] + additional_data]
    df_lagged = df.copy()
    for window in range(1, lags + 1):
        shifted = df.shift(window)
        shifted.columns = [x + "_lag" + str(window) for x in df.columns]

        df_lagged = pd.concat((df_lagged, shifted), axis=1)
    #df_lagged = df_lagged.fillna(method='ffill')
    df_lagged= df_lagged.interpolate(method='linear')
    df_lagged = df_lagged.dropna()
    #df_lagged = df_lagged.reindex(sorted(df_lagged.columns), axis=1)

    # print(df_lagged)
    # df_lagged[ticker+"_2labels"]=np.floor(df_lagged[ticker]/df_lagged[ticker+"_lag1"]).astype(int)

    # train_set
    df_filtered = df_lagged.loc[:train_date_last]
    # X_train=df_filtered.drop(columns=[ticker, ticker+"_2labels"])
    X_train = df_filtered.drop(columns=[ticker])
    # y_train=df_filtered[ticker+"_2labels"]
    y_train = df_filtered[ticker]

    # test set
    df_filtered = df_lagged.loc[test_date_start:test_date_last]
    # X_test=df_filtered.drop(columns=[ticker, ticker+"_2labels"])
    X_test = df_filtered.drop(columns=[ticker])
    # y_test=df_filtered[ticker+"_2labels"]
    y_test = df_filtered[ticker]

    # we convert to numpy array
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    return X_train, y_train, X_test, y_test, df_filtered


In [893]:



def initialize_data(days, lags, nb_predict_days):

    from datetime import datetime, timedelta



    # define main date in models for defining training and test sets dates.
    #yesterday = datetime.now() - timedelta(1)
    yesterday=datetime.now()
    start_date = yesterday - timedelta(days=days)
    train_date_start = start_date.strftime("%Y-%m-%d")
    train_date_last = yesterday - timedelta(days=nb_predict_days + 1)  # nombre de jours a predire
    train_date_last = train_date_last.strftime("%Y-%m-%d")

    test_date_start = yesterday - timedelta(days=nb_predict_days)
    test_date_start = test_date_start.strftime("%Y-%m-%d")
    test_date_last = yesterday.strftime("%Y-%m-%d")

    return yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last,days



In [894]:


def create_predict_set(ticker ,features ,lags, nb_predict_days, additional_data):

    yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last, days = initialize_data(
        lags * 2, lags, nb_predict_days)
    #  List of features X_train, y_train, X_test,y_test
    print("features shape in module create_predict_set")
    print(features.shape)

    df =features[[ticker ] +additional_data]
    print('df=features+additional data in module create_predict_set')
    print(df)
    
    #duplicate last row of features to use in the lag - this is to add tomorrow date in the predict set
    df_last=df.iloc[-1:]  #we get last date
    print(df_last.index[-1])
    print(type(df_last.index[-1]))
    #last_date=datetime.fromtimestamp(df_last.index[-1]) #index is of type timestamp therefore convert to datetime
    last_date=df_last.index[-1] #index is of type timestamp therefore convert to datetime
    df=df.append(df_last)  #we append last date to end
    df.index.array[-1] = last_date+timedelta(1) #we change to tomorrows date
    #df.set_axis((df.index[:-1].union([last_date],sort=False)),axis=0)
    


    
    
    df_lagged =df.copy()
    print("ici df lagged", df_lagged)
    for window in range(1, lags + 1):
        shifted = df.shift(window)
        shifted.columns = [x + "_lag" + str(window) for x in df.columns]

        df_lagged = pd.concat((df_lagged, shifted), axis=1)
    print('df_lagged of features+additional data before dropna in module create_predict_set')
    print(df_lagged.shape)
    print(df_lagged)
    #df_lagged = df_lagged.fillna(method='ffill')
    df_lagged= df_lagged.interpolate(method='linear')
    df_lagged.to_csv('df_lagged3.csv')
    df_lagged = df_lagged.dropna()
    print('df_lagged of features+additional data after dropna and before sort in module create_predict_set')
    print(df_lagged.shape)
    df_lagged =df_lagged.reindex(sorted(df_lagged.columns), axis=1)
    print('df_lagged of features+additional data after dropna in module create_predict_set')
    print(df_lagged.shape)
    print(df_lagged)
    # test set
    #df_filtered = df_lagged.loc[test_date_start:test_date_last]
    df_filtered=df_lagged
    
    print('df_filtered which is only start to last date of predict  in module create_predict_set')
    print(df_filtered.shape)
    
    X_test =df_filtered.drop(columns=[ticker])
    y_test =df_filtered[ticker]

    # we convert to numpy array
    X_test =X_test.to_numpy()
    y_test= y_test.to_numpy()
    return X_test ,y_test ,df_filtered







In [895]:
#import bamboolib as bam
import os
import os.path

import numpy as np
from yahoo_fin import stock_info as si

#from VBTT2_IO.IO import write_list, read_list,delete_then_get_model_from_bucket,read_config_file
import time



#functions to fetch and process SP500 data

def read_create_write_SP500(SP500_tickers,filename_json):
    # This create the complete master list of all tickers in SP500, their sectors, their industries
    # Check if SP500 json file exist first otherwise process SP500

    check=delete_then_get_model_from_bucket(filename_json) #delete local file and copy file from bucket to have fresh one
    #file_exists = os.path.exists(filename_json) #no longer required with blob check above

    if check==True:
        SP500_list = read_list(filename_json)
        SP500_list = np.array(SP500_list)  # we need an array
    else:
        SP500_list = read_write_SP500(SP500_tickers,filename_json)  # this will extract

    return SP500_list


def get_ticker_sector(ticker):
    # version 2
    # prerequisites is to import Yahoo_finance.stock_info
    # probleme avec cette methode c'est que industry est ligne 18 ou 19 *
    # prerequisites 2 is to import pandas as pd; import numpy as np
    filename_json="SP500.json"
    file_exists = os.path.exists(filename_json)
    if file_exists:  # file json exist
        SP500_list = read_list(filename_json)
        SP500_list = np.array(SP500_list)  # we need an array
        #now we should extract the sector which is column 2 for in each item of this 2D array
        ticker_sector=SP500_list[:, 1:2][SP500_list[0:, 0:1] == ticker].tolist()

        return ticker_sector[0]
    else:
        df = si.get_company_info(ticker)  # a utiliser pour trouver le secteur
        df = df.reset_index()
        sector = df.loc[df['Breakdown'].isin(['sector'])]  # from bamboolib to extract sector
        sector = sector.iloc[0, 1]  # this is to extract just the value
        industry = df.loc[df['Breakdown'].isin(['industry'])]
        industry = industry.iloc[0, 1]
        ticker_sector = []
        ticker_sector.append([ticker, sector, industry])
        return ticker_sector[0]


def read_write_SP500(tickers_list,filename_json):
    # Initialisation of SP500 data - find sector, industry for all tickers in SP500

    SP500_list = []
    for ticker in tickers_list:
        print("ticker in module read_write_SP500",ticker)
        SP500_list.append(get_ticker_sector(ticker))
        if len(SP500_list) % 30 == 0:
            time.sleep(30)
            # for some reason processing SP500 one shot is failing. so sleep of 20 seconds for each 50 tickers
    SP500_list = np.array(SP500_list)
    # Save in a file to reduce processing time next time
    write_list(SP500_list.tolist(), filename_json)  # this function work if it is a list
    return SP500_list  # this returns the array, not the list


def get_all_tickers_sector(sector):
    # prerequisites list is an array of ticker, sector and industry
    filename_json = "SP500.json"
    SP500_list = read_list(filename_json)
    SP500_list = np.array(SP500_list)  # we need an array


    sub_list_tickers = np.array(SP500_list)
    fltr = np.asarray([sector])
    result = sub_list_tickers[np.in1d(sub_list_tickers[:, 1], fltr)]
    return result[:, 0:1]
    # on veut extraire les tickers donc toutes les rangees (0:0) et la colonne 0 donc (0:1)


def get_all_tickers_industry(industry):
    # prerequisites list is an array of ticker, sector and industry
    filename_json = "SP500.json"
    SP500_list = read_list(filename_json)
    SP500_list = np.array(SP500_list)  # we need an array

    sub_list_tickers = np.array(SP500_list)
    fltr = np.asarray([industry])
    result = sub_list_tickers[np.in1d(sub_list_tickers[:, 2], fltr)]
    return result[:, 0:1]
    # on veut extraire les tickers donc toutes les rangees (0:0) et la colonne 0 donc (0:1)


def generate_enhanced_data(sector,ticker):
    additional_data = read_config_file()[1]
    sector_list=get_all_tickers_sector(sector)
    additional_data = np.concatenate((sector_list, additional_data), axis=None)
    additional_data=additional_data.tolist()
    additional_data.remove(ticker)
    return additional_data

In [896]:

test1=['a','b','c','c','b']
additional_data=['1','2','2']
additional_data=np.concatenate((test1, additional_data), axis=None)
additional_data=additional_data.tolist()
additional_data.remove('2')
additional_data


['a', 'b', 'c', 'c', 'b', '1', '2']

In [897]:
additional_data

['a', 'b', 'c', 'c', 'b', '1', '2']

In [898]:
pip install google.api.core

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [899]:
pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [900]:
from google.api_core.exceptions import NotFound

In [901]:

from google.cloud import storage

In [902]:
#import bamboolib as bam

# Input Output functions to save and read files

#import bamboolib as bam
import json
import configparser
import logging
import joblib

import os
import os.path




In [903]:
def write_list(a_list, filename_json):
    print("Started writing list data into a json file")
    with open(filename_json, "w") as fp:
        json.dump(a_list, fp)
        print("Done writing data into .json file")
    upload_file_to_bucket(filename_json)


# Read list to memory
def read_list(filename_json):
    # for reading also binary mode is important
    with open(filename_json, 'rb') as fp:
        n_list = json.load(fp)
        return n_list




def read_config_file():
    # return element that is in configfile. exemple additional data
    config = configparser.ConfigParser()
    config.read('config.ini')
    version = config['DEFAULT']['version']
    additional_data = config['DEFAULT']['additional_data'].split(',')
    regressor = config['DEFAULT']['regressor']
    model = config['DEFAULT']['model']
    root_bucket=config['DEFAULT']['gcloud_root_bucket']

    # ^TNX reasury yield is the annual return investors can expect from holding a U.S. government security with a given
    # ^GSPC tracks the performance of the stocks of 500 large-cap companies in the US"
    # CL=F crude oil pricesi.get_data(result[0][0])

    return version, additional_data,regressor, model, root_bucket




#file existimport logging


def create_bucket(bucket_name):
    log = logging.getLogger()

    storage_client = storage.Client()
    if bucket_name not in [x.name for x in storage_client.list_buckets()]:
        bucket = storage_client.create_bucket(bucket_name)

        log.info("Bucket {} created".format(bucket.name))
    else:
        log.info("Bucket {} already exists".format(bucket_name))



def get_bucket_name():
    root_bucket = read_config_file()[4]
    version = read_config_file()[0]
    bucket_name=f'{root_bucket}_{version.replace(".", "")}'
    create_bucket(bucket_name)
    return bucket_name


def upload_file_to_bucket(model_file_name):
    bucket_name=get_bucket_name()
    log = logging.getLogger()
    log.warning(f'uploading {model_file_name} to {bucket_name}')
    storage_client = storage.Client()
    bucket=storage_client.bucket(bucket_name)
    blob = bucket.blob(model_file_name)
    blob.upload_from_filename(model_file_name)


def delete_blob(blob_name):
    log = logging.getLogger()
    bucket_name = get_bucket_name()
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()
    log.warning(f'old blob {blob_name} deleted from {bucket_name}\n')


def delete_then_get_model_from_bucket(model_filename):
    log = logging.getLogger()
    if (os.path.exists(model_filename)):
        os.remove(model_filename)
        log.warning(f'old file {model_filename} deleted locally\n')

    bucket_name = get_bucket_name()
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob=bucket.blob(model_filename)

    try:
        blob.download_to_filename(model_filename)
        log.warning(f'new file   {model_filename} downloaded\n')
        return True #file download ok therefore retrain not required  or fileexists locally

    except NotFound as e:
        log.warning(f'file {model_filename} not found in bucket\n')
        return False  #retrain required or file does not exist locally

In [904]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from yahoo_fin import stock_info as si
import joblib

In [905]:
def Model_Train_Save(ticker_list_for_models, years, lags, additional_data, nb_predict_days):
    # Functions for model training and algorythm data and import

    # import the regressors
    version, additional_data, regressor, MODEL,bucket=read_config_file()
   

    if MODEL=='DecisionTree()':
        from sklearn.tree import DecisionTreeRegressor
        MODEL = DecisionTreeRegressor()
    elif MODEL=='LinearRegression()':
        from sklearn.linear_model import LinearRegression
        MODEL = LinearRegression()
    elif MODEL=='svm.SVR()':
        from sklearn import svm
        MODEL = svm.SVR()
    elif MODEL=="DecisionTree(max_depth=5)":
        from sklearn.tree import DecisionTreeRegressor
        MODEL = DecisionTreeRegressor(max_depth=5)
    elif MODEL=="Ridge(alpha=1.0)":
        from sklearn.linear_model import Ridge
        MODEL=Ridge(alpha=1.0)
    elif MODEL=="Lasso(alpha=1.0)":
        from sklearn.linear_model import Lasso
        MODEL=Lasso(alpha=1.0)
    elif MODEL=="xgboost":
        import xgboost as xgb
        MODEL= xgb.XGBClassifier()
    elif MODEL=="RandomForestRegressor(n_estimators=100)":   
        from sklearn.ensemble import RandomForestRegressor
        MODEL=RandomForestRegressor(n_estimators=100)
    else:
        #MODEL=='LinearRegression'
        from sklearn.linear_model import LinearRegression
        MODEL = LinearRegression()
        






    #### Initialisation of variables and data
    # the timeframe for training and test sets and predict
    days = 360 * years  # Nunber of days in the model
    yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last, days = initialize_data(
        days,
        lags,
        nb_predict_days)
    SP500_tickers = si.tickers_sp500()  # get list of tickers

    # print(f"SP500_tickers = {SP500_tickers}")



    # read master list of ticker, sector, industries or if not found, create it and save it#
    SP500_list = read_create_write_SP500(SP500_tickers, "SP500.json")

    ### validation if we have everything
    # print(f"Input ->years {years}")
    # print(f"Input ->ticker list {ticker_list_for_models}")
    # print(f"Input->lags {lags}")
    # print(f"Input ->predict days {nb_predict_days}")
    # print(f"Period->yesterday {yesterday}")
    # print(f"period->train_date_start {train_date_start}")
    # print(f"Period->train_date_last {train_date_last}")
    # print(f"Period->test_date_start {test_date_start}")
    # print(f"Period->test_date_last {test_date_last}")

    # print(f"This is the tickers for our model {ticker_list_for_models}")
    # print(f"This is the additional data  we add to the tickers for the model {additional_data}")
    # print(f"VALIDATE - This is the number of training days of the train dataset {days}")

    # Get features
    #matrix_features_sector = preprocessing(ticker_list_for_models, additional_data, days)

    #### Run models for all tickers selected in input  and predict

    predictions = pd.DataFrame()  # to store predictions

    for ticker in ticker_list_for_models:
        sector = get_ticker_sector(ticker)
        additional_data = read_config_file()[1]
        additional_data = generate_enhanced_data(sector,ticker)
        print("in module model_Train_Save, additional_data")
        print(additional_data)

        matrix_features_sector = preprocessing(ticker, additional_data, days)

        X_train, y_train, X_test, y_test, df_filtered = create_train_test_set(ticker, matrix_features_sector, lags,additional_data,days,nb_predict_days)
        
        print("in module model_Train_Save, print model")
        print(MODEL)
        print("in module model_Train_Save, print ticker")
        print(ticker)
        MODEL.fit(X_train, y_train)
        # save the model to disk
        filename = ticker + '_model.sav'
        joblib.dump(MODEL, filename)
        upload_file_to_bucket(filename)
        temp_pred = model_predict(MODEL, ticker, X_test, y_test)

        predictions = predictions.append(temp_pred, ignore_index=True)  # this is to store in the master pandas list

    # add binary buy=1 and sell=0
    df_lagged = add_buy_sell_to_prediction(predictions,ticker_list_for_models)
    df_lagged  # lag_lagged is a DF containing predictions + buy and Sell label

    ticker = "*all*"
    accuracy = balanced_accuracy(ticker, df_lagged)
    print(f"Accuracy score for {ticker} is {accuracy}.")

    accuracy = []
    for ticker in ticker_list_for_models:
        accuracy.append([balanced_accuracy(ticker, df_lagged), ticker])
    DF_accuracy = pd.DataFrame(accuracy, columns=["Blc accuracy", "Ticker"])
    DF_accuracy



In [906]:

def balanced_accuracy(ticker, predict):
    # put *all* to have global accuracy score for all the predictions
    # or put a ticker name
    if ticker == '*all*':
        return balanced_accuracy_score(predict['y_testb'], predict['y_predb'])
    else:
        return balanced_accuracy_score(predict['y_testb'][predict['ticker'] == ticker],
                                       predict['y_predb'][predict['ticker'] == ticker])


In [907]:


def model_predict(MODEL, ticker, X, y):
    print('X test shape in module model_predict',X.shape)
    y_pred = MODEL.predict(X)
    temp_pred = predictions_compile(y, y_pred, ticker)  # this is to store temporary ytest, ypredict, ticker
    return temp_pred


In [908]:


def predictions_compile(y_test, y_pred, ticker):
    print("module predictions compile")
    print("y_test_shape in module predictions compile",y_test.shape)
    print("y_pred_shape in module predictions compile",y_pred.shape)
   
    print ("y_test in module predictions compile", y_test)
    print ("y_pred in module predictions compile",y_pred)
    print ("ticker in module predictions compile",ticker)
    # this allow to create a dataframe of y_test, y_predict for a given ticker
    predict_df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
    predict_df['ticker'] = ticker
    print ("predict_df in module predictions compile",predict_df)
    return predict_df
    # we need to have an initial empty dataframe to store the predictions



In [909]:


def add_buy_sell_to_prediction(predictions, ticker_list_for_models):
    # this section is to calculate the label=buy or sell
    # it is adding colum y_testb and y_predictb to data frame predictions
    # buy=1
    # sell=0

    df_lagged = pd.DataFrame()
    for ticker in ticker_list_for_models:
        df = predictions[predictions['ticker'] == ticker]
        df_lagged_ticker = df.copy()
        for window in range(1, 1 + 1):
            shifted = df.shift(window)
            shifted.columns = [x + "_lag" + str(window) for x in df.columns]

            df_lagged_ticker = pd.concat((df_lagged_ticker, shifted), axis=1)
            df_lagged_ticker = df_lagged_ticker.dropna()
            df_lagged_ticker['y_testb'] = np.floor(df_lagged_ticker['y_test'] / df_lagged_ticker['y_test_lag1']).astype(
                int)
            df_lagged_ticker['y_predb'] = np.floor(df_lagged_ticker['y_pred'] / df_lagged_ticker['y_pred_lag1']).astype(
                int)
            # *** using map function to decide buy or sell
            category = {1: "Buy", 0: "Sell"}
            df_lagged_ticker['y_recommend'] = df_lagged_ticker['y_predb'].map(category)
            df_lagged_ticker['daily return %'] = (
                        (df_lagged_ticker['y_test'] / df_lagged_ticker['y_test_lag1']) - 1).where \
                (df_lagged_ticker['y_predb'] == 1, \
                 (((df_lagged_ticker['y_test_lag1'] / df_lagged_ticker['y_test']) - 1)))
            df_lagged_ticker['daily return %'] = df_lagged_ticker['daily return %'] * 100
        df_lagged = df_lagged.append(df_lagged_ticker)

    return df_lagged


In [910]:
# functions to predict model and calculate accuracy

import os
# import bamboolib as bam
import os.path

import joblib
import numpy as np
import pandas as pd

In [911]:
def YF_datetime():
    date_predict=datetime.now()
    if date_predict.hour>=17:
        date_predict=date_predict+timedelta(1)
    return date_predict

In [912]:
def predict_ticker(ticker_list_for_models):
    ticker_list_for_models = ticker_list_for_models.split('-')
    # note when we use flask we will use /ticker/aapl-nflx-cdw
    # so we need to split

    # print(f"Example long_test: {long_test}")
    # print(f"Example short_test: {short_test}")
    # print(f"Example not_working: {not_working}")
    # ticker_list_for_models=input_ticker

    # initialisation model
    # how many years for the model
    years = 6
    lags = 30  # how many days of lags we need of this model, this is like an hyperparameter for us
    version, additional_data, regressor, model,bucket = read_config_file()  # other data needed
    print(version)
    print(regressor)
    print(model)
    print(additional_data)
    print(bucket)

    nb_predict_days = 30  # size of test data in number of days

    # if we don't have model for a ticker in list, retrain model and save
    check = True
    for ticker in ticker_list_for_models:
        check=delete_then_get_model_from_bucket(ticker + "_model.sav") #this download model from bucket. Model will be trained if model does not exist

        if not (os.path.exists(ticker + "_model.sav")):
            check = False
            break  # this allow to continue and not go through the list if file not exist

    if check == False:
        #Retrain all for the select list of tickers
        print(f"Training model for at least one ticker in {ticker_list_for_models}")
        Model_Train_Save(ticker_list_for_models, years, lags, additional_data, nb_predict_days)
    else:
        print(f"no model Training is needed for  {ticker_list_for_models}")
        ## get variable for start the prediction
        ##just in case, we fetch 2 time the lags so that we don't have issue when lagging

    yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last, days = initialize_data(
        lags * 2, lags, nb_predict_days)

    ### validation if we have everything
    print(f"Input ->years {years}")
    print(f"Input ->ticker list {ticker_list_for_models}")
    print(f"Input->lags {lags}")
    print(f"Input ->predict days {nb_predict_days}")
    print(f"Period->yesterday {yesterday}")
    print(f"period->train_date_start {train_date_start}")
    print(f"Period->train_date_last {train_date_last}")
    print(f"Period->test_date_start {test_date_start}")
    print(f"Period->test_date_last {test_date_last}")

    print(f"This is the tickers for our model {ticker_list_for_models}")
    print(f"This is the additional data  we add to the tickers for the model {additional_data}")

    #df_tomorrow = preprocessing(ticker_list_for_models, additional_data, lags * 2)  # add additional features
    #df_tomorrow.shape

    # to store predictions
    Predictions = pd.DataFrame()

    for ticker in ticker_list_for_models:
        # load the saved model for the ticker
        filename = ticker + "_model.sav"
        loaded_model = joblib.load(filename)
        sector=get_ticker_sector(ticker)
        additional_data=read_config_file()[1]
        additional_data=generate_enhanced_data(sector,ticker)
        print(additional_data)
        df_tomorrow = preprocessing(ticker, additional_data, lags * 2)  # add additional features
        print("predict df_tomorrow shape before creating predict set")
        print(df_tomorrow.shape)

        X_test, y_test, df_filtered = create_predict_set(ticker, df_tomorrow, lags, nb_predict_days,
                                                         additional_data)  # this is X and y

        print("X_test shape in module predict_ticker before calling model_predict and after creating predict_set",X_test.shape)


        temp_pred = model_predict(loaded_model, ticker, X_test, y_test)
        print("temp_pred shape in module predict_ticker after calling model_predict",temp_pred.shape)

        print(temp_pred)
        # ===
        temp_pred.drop(labels=[0], inplace=True)
        temp_pred.to_csv('temp_pred.csv')

        from datetime import timedelta

        df_filtered2 = pd.DataFrame(df_filtered[ticker])

        df_filtered2 = df_filtered2.reset_index()

        # Deleted 1 row in df_filtered2
        df_filtered2.drop(labels=[0], inplace=True)

        # Renamed columns Date
        df_filtered2.rename(columns={'index': 'Date'}, inplace=True)

        df_filtered2['Predicted for'] = df_filtered2['Date'] 
        # df_filtered2['Prediction for']=df_filtered2['Date']
        df_filtered2 = df_filtered2[['Date', 'Predicted for']]

        # Step: Copy a dataframe column

        temp_pred2 = pd.concat([temp_pred, df_filtered2], axis=1)

        # ===

        Predictions = Predictions.append(temp_pred2, ignore_index=True)  # this is to store in the master pandas list

        # print(f"temp_pred: \n{temp_pred}")
        # print(f"temp_pred2: \n{temp_pred2}")
        # print(f"temp_pred2: \n{temp_pred2}")
        # print(f"df_filtered2: \n{df_filtered2}")

        # print(f"Predictions:\n {Predictions}")

    # adding binary buy or sell to predictions dataframe
    Predictions = add_buy_sell_to_prediction(Predictions,ticker_list_for_models)
    Predictions.to_csv('predictions.csv')

    ticker = "*all*"
    accuracy_all= balanced_accuracy(ticker, Predictions)
    print(f"Accuracy score for {ticker} is {accuracy_all}.")
    # provide a data frame of the accuracies

    avg_return = [0]  # render for view html need a first element to be 0
    for ticker in ticker_list_for_models:
        ticker_return = round(Predictions['daily return %'][Predictions['ticker'] == ticker].iloc[:-1].mean(), 2)
        avg_return.append(ticker_return)

    DF_Recommendations = []
    for ticker in ticker_list_for_models:
        DF_Recommendations.append([ticker, "", "", "", balanced_accuracy(ticker, Predictions.iloc[:-1])])
        Recommendations = pd.DataFrame(DF_Recommendations,
                                       columns=["Ticker", 'Predicted for', 'Predicted', "Recommended", "Accuracy"])

    # ********************************************************
    # suggestion - DF_accuracy change to DF_accuracy_recommendation
    # suggestion - resultat change as follow: Date, Observed Value, Date Prediction, Predicted Value,Recommendation
    # suggestion - now date observed value should be change to NA
    # ********************************************************

    for ticker in ticker_list_for_models:
        # results = Predictions[['y_test', 'y_pred', 'ticker', 'y_predb','y_recommend']][Predictions['ticker'] == ticker]
        results = Predictions[['y_test', 'y_pred', 'ticker', 'y_predb', 'y_recommend','daily return %']][Predictions['ticker'] == ticker]
        date_predict=YF_datetime() #can be confusing but yesterday is datetime.now. good thing is to take latest in results +1
        date_predict = date_predict.strftime("%Y/%m/%d")
        ticker_predicted = results.iloc[-1]['y_pred']  # this is the last row containing result
        ticker_recommend = results.iloc[-1]['y_recommend']  # this is the last row containing result
        Recommendations.loc[Recommendations['Ticker'] == ticker, 'Predicted for'] = date_predict  # to change content of a cell
        Recommendations.loc[Recommendations['Ticker'] == ticker, 'Predicted'] = ticker_predicted
        Recommendations.loc[Recommendations['Ticker'] == ticker, 'Recommended'] = ticker_recommend

        Predictions.loc[(Predictions['Date']==date_predict) & (Predictions['ticker']==ticker),'y_test']=np.nan
        Predictions.loc[(Predictions['Date']==date_predict) & (Predictions['ticker']==ticker),'daily return %']=np.nan
           

        # print(f"Prediction for {yesterday+timedelta(1)} -- ticker: {ticker} {'**resultat**'}\n {resultat.tail()}\n\n")

    Results = Predictions[['ticker', 'Date', 'y_test', 'Predicted for', 'y_pred', 'y_recommend','daily return %']]
    Results = Results.rename(
        columns={'ticker': 'Ticker', 'y_test': 'Observed', 'y_pred': 'Predicted', 'y_recommend': 'Recommended','daily return %':'Daily return %'})


    return Results, Recommendations,avg_return



In [913]:
import os
import os.path
import numpy as np
from yahoo_fin import stock_info as si
#from flask import *

In [914]:
def get_sp500():
    SP500_tickers = si.tickers_sp500()
    delete_blob("SP500.json")
    SP500_list = read_create_write_SP500(SP500_tickers, "SP500.json")
    return f"JSON was generated  successfully- try again with /ticker/xxx or /details/xxx (where xxx is your list of tickers separated by '-'."

In [915]:
#get_sp500()

In [916]:

def check_ticker(ticker):
    ticker = ticker.upper()
    version=read_config_file()[0]
    model_html=read_config_file()[3]
    filename_json="SP500.json"
    check=delete_then_get_model_from_bucket(filename_json)  # delete local file and copy file from bucket to have fresh one
    #file_exists = os.path.exists("SP500.json") # no longer required with blob check above
    if check==True:  #file json exist
        SP500_list = read_list(filename_json)
        SP500_list = np.array(SP500_list)  # we need an array
        validation=all([([x] in SP500_list[:,:1]) for x in ticker.split("-")])
        # all allow to check if a list o bolean is tru or false . all([true, false, true....etc])
        # X in SP50_list, etc.... .... will check if x is in my SP500 list. here all row and column 0
        # for x in is selecting each at a time

        if validation==True:
            Results, Recommendations,avg_return = predict_ticker(ticker)
            print("avg return",avg_return)
            return Results, Recommendations,avg_return
            

        else:
            return f"Incorrect ticker, please fix or select another."
    else:
        return f"JSON does not exists - Please generate JSON  with /get_SP500"


In [917]:
ticker='CDW'
results,recommendations,avg_return=check_ticker(ticker)

old file SP500.json deleted locally

new file   SP500.json downloaded

old file CDW_model.sav deleted locally



0.1
*
Lasso(alpha=1.0)
['^tnx', '^GSPC', 'CL=F']
stock-363101


new file   CDW_model.sav downloaded



no model Training is needed for  ['CDW']
Input ->years 6
Input ->ticker list ['CDW']
Input->lags 30
Input ->predict days 30
Period->yesterday 2022-11-09 07:36:41.372472
period->train_date_start 2022-09-10
Period->train_date_last 2022-10-09
Period->test_date_start 2022-10-10
Period->test_date_last 2022-11-09
This is the tickers for our model ['CDW']
This is the additional data  we add to the tickers for the model ['^tnx', '^GSPC', 'CL=F']
['AAPL', 'ACN', 'ADBE', 'ADI', 'ADSK', 'AKAM', 'AMAT', 'AMD', 'ANET', 'ANSS', 'APH', 'AVGO', 'BR', 'CDAY', 'CDNS', 'CRM', 'CSCO', 'CTSH', 'DXC', 'ENPH', 'EPAM', 'FFIV', 'FIS', 'FISV', 'FLT', 'FTNT', 'FTV', 'GLW', 'GRMN', 'HPE', 'HPQ', 'IBM', 'INTC', 'INTU', 'IT', 'JKHY', 'JNPR', 'KEYS', 'KLAC', 'LDOS', 'LRCX', 'MCHP', 'MPWR', 'MSFT', 'MSI', 'MU', 'NLOK', 'NOW', 'NTAP', 'NVDA', 'NXPI', 'ON', 'ORCL', 'PAYC', 'PTC', 'QCOM', 'QRVO', 'SEDG', 'SNPS', 'STX', 'SWKS', 'TDY', 'TEL', 'TER', 'TRMB', 'TXN', 'TYL', 'VRSN', 'WDC', 'ZBRA', '^tnx', '^GSPC', 'CL=F']
pre

/tmp/ipykernel_1846559/1130624236.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df_last)  #we append last date to end


df_lagged of features+additional data after dropna and before sort in module create_predict_set
(13, 2294)
df_lagged of features+additional data after dropna in module create_predict_set
(13, 2294)
                  AAPL   AAPL_lag1  AAPL_lag10  AAPL_lag11  AAPL_lag12  \
2022-10-24  149.449997  147.270004  140.419998  140.089996  145.429993   
2022-10-25  152.339996  149.449997  138.979996  140.419998  140.089996   
2022-10-26  149.350006  152.339996  138.339996  138.979996  140.419998   
2022-10-27  144.800003  149.350006  142.990005  138.339996  138.979996   
2022-10-28  155.740005  144.800003  138.380005  142.990005  138.339996   
2022-10-31  153.339996  155.740005  142.410004  138.380005  142.990005   
2022-11-01  150.649994  153.339996  143.750000  142.410004  138.380005   
2022-11-02  145.029999  150.649994  143.860001  143.750000  142.410004   
2022-11-03  138.880005  145.029999  143.389999  143.860001  143.750000   
2022-11-04  138.380005  138.880005  147.270004  143.389999  14

/tmp/ipykernel_1846559/2120334667.py:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Predictions = Predictions.append(temp_pred2, ignore_index=True)  # this is to store in the master pandas list
/tmp/ipykernel_1846559/1752493272.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lagged = df_lagged.append(df_lagged_ticker)


In [918]:
results

,Ticker,Date,Observed,Predicted for,Predicted,Recommended,Daily return %
1,CDW,2022-10-26,167.580002,2022-10-26,167.523064,Buy,-0.581393
2,CDW,2022-10-27,168.429993,2022-10-27,167.758479,Buy,0.507215
3,CDW,2022-10-28,172.880005,2022-10-28,173.634578,Buy,2.642055
4,CDW,2022-10-31,172.809998,2022-10-31,173.026751,Sell,0.040511
5,CDW,2022-11-01,172.580002,2022-11-01,172.373300,Sell,0.133269
...,...,...,...,...,...,...,...
7,CDW,2022-11-03,172.830002,2022-11-03,170.167945,Buy,0.249425
8,CDW,2022-11-04,172.410004,2022-11-04,173.288040,Buy,-0.243012
9,CDW,2022-11-07,172.309998,2022-11-07,173.980210,Buy,-0.058005
10,CDW,2022-11-08,175.720001,2022-11-08,174.041127,Buy,1.978994


In [919]:
recommendations

,Ticker,Predicted for,Predicted,Recommended,Accuracy
0,CDW,2022/11/09,175.364998,Buy,0.75


## New section to see enhanced date and preprocessed data

In [920]:
ticker='GOOG'
sector=get_ticker_sector(ticker)
additional_data=read_config_file()[1]
additional_data=generate_enhanced_data(sector,ticker)

In [921]:
data=preprocessing(ticker,additional_data,10)

preprocing module ticker GOOG
preprocessing module additional data ['ATVI', 'CHTR', 'CMCSA', 'DIS', 'DISH', 'EA', 'FOX', 'FOXA', 'GOOGL', 'IPG', 'LUMN', 'LYV', 'META', 'MTCH', 'NFLX', 'NWS', 'NWSA', 'OMC', 'PARA', 'T', 'TMUS', 'TTWO', 'VZ', 'WBD', '^tnx', '^GSPC', 'CL=F']
preprocessing module days 10


In [922]:
SP500_list = read_list("SP500.json")
#sub_list_tickers = np.array(SP500_list)
fltr = np.asarray("Communication Services")
sector = sub_list_tickers[np.in1d(sub_list_tickers[:, 1], fltr)].tolist()
sector

[['ATVI', 'Communication Services', 'Electronic Gaming & Multimedia'],
 ['CHTR', 'Communication Services', 'Telecom Services'],
 ['CMCSA', 'Communication Services', 'Telecom Services'],
 ['DIS', 'Communication Services', 'Entertainment'],
 ['DISH', 'Communication Services', 'Telecom Services'],
 ['EA', 'Communication Services', 'Electronic Gaming & Multimedia'],
 ['FOX', 'Communication Services', 'Entertainment'],
 ['FOXA', 'Communication Services', 'Entertainment'],
 ['GOOG', 'Communication Services', 'Internet Content & Information'],
 ['GOOGL', 'Communication Services', 'Internet Content & Information'],
 ['IPG', 'Communication Services', 'Advertising Agencies'],
 ['LUMN', 'Communication Services', 'Telecom Services'],
 ['LYV', 'Communication Services', 'Entertainment'],
 ['META', 'Communication Services', 'Internet Content & Information'],
 ['MTCH', 'Communication Services', 'Internet Content & Information'],
 ['NFLX', 'Communication Services', 'Entertainment'],
 ['NWS', 'Communica

In [923]:
Xy=get_yf_dataframe(data,10)
Xy

,ATVI,CHTR,CL=F,CMCSA,DIS,DISH,EA,FOX,FOXA,GOOG,...,NWSA,OMC,PARA,T,TMUS,TTWO,VZ,WBD,^GSPC,^tnx
2022-10-31,72.800003,367.619995,86.529999,31.740000,106.540001,14.91,125.959999,27.200001,28.870001,94.660004,...,16.870001,72.750000,18.320000,18.230000,151.559998,118.480003,37.369999,13.00,3871.979980,4.077
2022-11-01,73.309998,357.000000,88.370003,31.559999,106.000000,15.06,126.269997,28.610001,30.530001,90.500000,...,16.990000,73.099998,19.170000,18.350000,152.279999,116.309998,37.369999,13.35,3856.100098,4.052
2022-11-02,72.660004,347.250000,90.000000,30.910000,101.820000,14.55,128.660004,28.170000,30.049999,87.070000,...,16.469999,71.629997,16.790001,18.430000,150.020004,113.550003,37.709999,12.68,3759.689941,4.059
2022-11-03,71.900002,344.250000,88.169998,30.379999,99.250000,13.72,126.620003,28.020000,29.950001,83.489998,...,16.459999,71.139999,16.209999,18.170000,148.169998,111.300003,37.150002,11.97,3719.889893,4.124
2022-11-04,71.959999,348.820007,92.610001,31.000000,99.580002,13.35,129.759995,27.830000,29.840000,86.699997,...,16.680000,72.220001,15.570000,18.320000,148.830002,108.529999,37.240002,10.43,3770.550049,4.156
2022-11-07,71.099998,353.279999,91.790001,31.600000,100.430000,14.67,132.419998,27.440001,29.139999,88.650002,...,16.719999,72.860001,16.370001,18.360001,149.779999,108.400002,37.189999,10.27,3806.800049,4.214
2022-11-08,72.000000,354.660004,88.910004,31.900000,99.900002,15.34,129.100006,27.620001,29.340000,88.910004,...,17.090000,74.099998,16.580000,18.570000,149.899994,93.570000,37.869999,10.06,3828.110107,4.126
